<a href="https://colab.research.google.com/github/luistorres109/ciencia_dados_ia_trabalho_final/blob/main/reconhecimento_irregularidades/trabalho_final_luis_ds_ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# AVISO LEGAL!!!
# Isto se trata de um trabalho acadêmico e não está sendo feita nenhuma acusação de fato se há alguma ilegalidade ou não nas licitações, e quem tem que decidir isso é o MP e a Justiça Brasileira
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from google.colab import drive
import pandas as pd

# Função para avaliar modelo
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, mse, r2

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
df = pd.read_csv("/content/drive/MyDrive/editais_dados.csv", delimiter=';')
print(df)

           Data Horario    Edital  \
0    2023-05-02     NaN  029/2023   
1    2023-05-10     NaN  135/2023   
2    2023-05-22     NaN  042/2023   
3    2023-05-22   16:00  138/2023   
4    2023-05-22   15:00  140/2023   
..          ...     ...       ...   
146  2023-08-23   16:00  297/2023   
147  2023-08-24   14:00  244/2023   
148  2023-08-28   10:10  301/2023   
149  2023-08-30   16:00  283/2023   
150  2023-09-05   08:40  294/2023   

                                                 Local  \
0                                                  NaN   
1                                                  NaN   
2                                                  NaN   
3                                         licitações-e   
4                                         licitações-e   
..                                                 ...   
146  Setor de Compras e Licitações na Prefeitura Mu...   
147  Setor de Compras e Licitações na Prefeitura Mu...   
148  Setor de Compras e Licitaçõe

In [14]:
# Substituir vírgulas por pontos e converter a coluna "Montante" para valores float
df['Montante'] = df['Montante'].replace('\.', '', regex=True)
df['Montante'] = df['Montante'].str.replace(',', '.').astype(float)

# Separar as características (features) e o alvo (target)
df_features = df.drop(["Montante", "Link da Página", "Horario", "Local"], axis=1)
df_target = df['Montante']

# Separar variáveis categóricas
categorical_cols = ['Data', 'Edital', 'Modalidade', 'Forma julgamento', 'Unidade gestora', 'Secretária', 'Objeto', 'Tipo']  # Substitua com suas colunas categóricas

# Inicializar o codificador de categorias
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Ajustar e transformar as variáveis categóricas
df_encoded = encoder.fit_transform(df_features[categorical_cols])

# Obter os nomes das colunas codificadas
encoded_columns = [f"{col}_{val}" for col, values in zip(categorical_cols, encoder.categories_) for val in values]

# Criado DF com as colunas codificadas
df_features_encoded = pd.DataFrame(df_encoded, columns=encoded_columns)

# Dividir os dados em treino, validação e teste (70-10-20)
X_train, X_temp, y_train, y_temp = train_test_split(df_features_encoded, df_target, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.66, random_state=42)

# Inicializar o modelo Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Treinar o modelo
rf_model.fit(X_train, y_train)

# Fazer previsões em partes do conjunto de dados
y_val_pred = rf_model.predict(X_val)
y_test_pred = rf_model.predict(X_test)

In [15]:
# Avaliar o modelo
test_rf_metrics_test = evaluate_model(y_test, y_test_pred)
val_rf_metrics_val = evaluate_model(y_val, y_val_pred)

print("\nTeste - Random Forest:")
print("- MAE (Mean Absolute Error):", test_rf_metrics_test[0])
print("- MSE (Mean Squared Error):", test_rf_metrics_test[1])
print("- R2 Score:", test_rf_metrics_test[2])

print("\nValidação - Random Forest:")
print("- MAE (Mean Absolute Error):", val_rf_metrics_val[0])
print("- MSE (Mean Squared Error):", val_rf_metrics_val[1])
print("- R2 Score:", val_rf_metrics_val[2])


Teste - Random Forest:
- MAE (Mean Absolute Error): 751776.9460161291
- MSE (Mean Squared Error): 2835669182738.448
- R2 Score: -0.38318891116702036

Validação - Random Forest:
- MAE (Mean Absolute Error): 593113.8498866666
- MSE (Mean Squared Error): 1870217771657.7346
- R2 Score: -23.523422703992406


In [16]:
# Fazer previsões em todo o conjunto de dados
df['Montante_Pred'] = rf_model.predict(df_features_encoded)

# Calcular a diferença entre os valores reais e previstos
df['Desvio'] = (df['Montante'] - df['Montante_Pred']) / df['Montante']

# Definição de um limiar para identificar desvios significativos
limiar_desvio = 0.25

# Filtrar licitações com desvios significativos
df = df[abs(df['Desvio']) > limiar_desvio].drop(['Data', 'Horario', 'Local', 'Modalidade', 'Forma julgamento', 'Unidade gestora', 'Secretária', 'Objeto', 'Tipo', 'Link da Página'], axis=1)
df = df.loc[df['Desvio'] >= 0]

# Visualizar as licitações irregulares
print("Dataframe com os editais (SUPOSTAMENTE) irregulares:")
print(df)
print(f"\nO numero total dessas licitacoes eh: {df[df.columns[0]].count()}")

Dataframe com os editais (SUPOSTAMENTE) irregulares:
       Edital     Montante  Montante_Pred    Desvio
0    029/2023    464297.40   3.357965e+05  0.276764
2    042/2023  10556017.14   6.523905e+06  0.381973
6    143/2023    398000.00   2.877282e+05  0.277065
8    009/2023    225000.00   1.478850e+05  0.342733
11   009/2023    225000.00   1.425740e+05  0.366338
16   186/2023    108648.00   4.282998e+04  0.605791
18   147/2023    128013.00   7.871448e+04  0.385106
22   157/2023   1434000.00   3.727548e+04  0.974006
23   168/2023   3845378.86   2.568397e+06  0.332082
27   048/2023    546891.74   1.821216e+05  0.666988
29   141/2023   1133354.54   5.687709e+04  0.949815
30   022/2023   7428000.00   1.781236e+05  0.976020
32   163/2023    156030.15   5.322291e+04  0.658893
35   166/2023    747193.50   5.177465e+05  0.307078
36   165/2023    230395.00   9.933154e+04  0.568864
43   056/2023    754670.77   5.450848e+05  0.277718
54   175/2023    432619.00   2.893832e+05  0.331090
63   196/20